## Vacationpy Homework


In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
vacation_data = pd.read_csv("data\weather_data.csv", index_col=0)
vacation_data.head(10)

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Windspeed (mph),Date
0,Chui,UY,-33.6971,-53.4616,58.21,87,0,9.33,1667785416
1,Ancud,CL,-41.8697,-73.8203,48.90,87,9,3.51,1667785416
2,Sinnamary,GF,5.3833,-52.9500,77.79,87,100,7.18,1667785416
3,Puerto Quijarro,BO,-17.7833,-57.7667,74.14,32,0,3.67,1667785417
4,Oga,IT,46.4631,10.3470,35.02,56,40,2.46,1667785417
5,Kruisfontein,ZA,-34.0033,24.7314,63.88,84,99,3.22,1667785418
6,Saskylakh,RU,71.9167,114.0833,-23.76,100,50,2.71,1667785418
7,Huilong,CN,31.8111,121.6550,65.93,61,0,11.48,1667785418
8,Buala,SB,-8.1450,159.5921,81.75,80,87,10.80,1667785419
9,Norman Wells,CA,65.2820,-126.8329,3.24,85,20,4.61,1667785419


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [43]:
# Heatmap of humidity
locations = vacation_data[["Latitude", "Longitude"]]
humidity = vacation_data["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
plt.savefig('images\Humidity_Heatmap.png')

fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 640x480 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [67]:
new_vacation_data = vacation_data[(vacation_data["Windspeed (mph)"] <= 10) & (vacation_data["Cloudiness"] <=30) & \
                                    (vacation_data["Max Temperature (F)"] >= 80) & (vacation_data["Max Temperature (F)"]<=100)].dropna()
new_vacation_data.dropna(inplace=True)
new_vacation_data

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Windspeed (mph),Date
20,Alyangula,AU,-13.8483,136.4192,86.68,62,21,8.10,1667785423
39,Kavaratti,IN,10.5669,72.6420,80.87,75,4,3.69,1667785429
73,Acapulco de Juárez,MX,16.8634,-99.8901,80.38,85,8,6.44,1667785440
79,Porto-Novo,BJ,6.4965,2.6036,82.22,85,29,3.65,1667785443
107,Ambon City,ID,-3.6954,128.1814,89.74,62,20,5.75,1667785454
231,Gawler,AU,-34.6000,138.7333,85.21,30,0,6.76,1667785498
241,Mareeba,AU,-17.0000,145.4333,88.45,55,4,7.29,1667785502
270,Port Augusta,AU,-32.5000,137.7667,86.70,18,0,5.06,1667785514
390,Dhidhdhoo,MV,6.8833,73.1000,80.73,68,7,3.69,1667785557
430,Kuching,MY,1.5500,110.3333,83.12,94,20,1.99,1667785573


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [68]:
# Create hotel df
hotel_df = new_vacation_data.loc[:, ['City', 'Country', 'Latitude', 'Longitude', 'Max Temperature (F)', 'Humidity', 'Cloudiness', 'Windspeed (mph)']]

# Add new column for Hotel Name to hotel_df
hotel_df['Hotel Name'] = ''
hotel_df


,City,Country,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Windspeed (mph),Hotel Name
20,Alyangula,AU,-13.8483,136.4192,86.68,62,21,8.10,
39,Kavaratti,IN,10.5669,72.6420,80.87,75,4,3.69,
73,Acapulco de Juárez,MX,16.8634,-99.8901,80.38,85,8,6.44,
79,Porto-Novo,BJ,6.4965,2.6036,82.22,85,29,3.65,
107,Ambon City,ID,-3.6954,128.1814,89.74,62,20,5.75,
231,Gawler,AU,-34.6000,138.7333,85.21,30,0,6.76,
241,Mareeba,AU,-17.0000,145.4333,88.45,55,4,7.29,
270,Port Augusta,AU,-32.5000,137.7667,86.70,18,0,5.06,
390,Dhidhdhoo,MV,6.8833,73.1000,80.73,68,7,3.69,
430,Kuching,MY,1.5500,110.3333,83.12,94,20,1.99,


In [89]:
import json

# Select needed columns from hotel_df
hotel_df = new_vacation_data.loc[:, ["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ''
hotel_df



,City,Country,Latitude,Longitude,Hotel Name
20,Alyangula,AU,-13.8483,136.4192,
39,Kavaratti,IN,10.5669,72.6420,
73,Acapulco de Juárez,MX,16.8634,-99.8901,
79,Porto-Novo,BJ,6.4965,2.6036,
107,Ambon City,ID,-3.6954,128.1814,
231,Gawler,AU,-34.6000,138.7333,
241,Mareeba,AU,-17.0000,145.4333,
270,Port Augusta,AU,-32.5000,137.7667,
390,Dhidhdhoo,MV,6.8833,73.1000,
430,Kuching,MY,1.5500,110.3333,


In [90]:
# create base URL for API calls to Google Maps
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "hotel",
            "keyword": "hotel",
            "radius": 5000,
            "key": g_key}

for index, row in hotel_data.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]

    params["location"] = f"{lat},{lng}"

    print(f'Retrieving results for Index {index}: {city_name}.')
    response = requests.get(base_url, params=params).json()
    results = response['results']

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print(f'No results found.....')
        
print(f'------- End of Search ---------')

Retrieving results for Index 20: Alyangula.
Closest hotel in Alyangula is Groote Eylandt Lodge.
Retrieving results for Index 39: Kavaratti.
Closest hotel in Kavaratti is Bismillah Hotel And Lodging.
Retrieving results for Index 73: Acapulco de Juárez.
Closest hotel in Acapulco de Juárez is Hotel Krystal Beach Acapulco.
Retrieving results for Index 79: Porto-Novo.
Closest hotel in Porto-Novo is Hotel Les Oliviers.
Retrieving results for Index 107: Ambon City.
Closest hotel in Ambon City is Swiss-Belhotel Ambon.
Retrieving results for Index 231: Gawler.
Closest hotel in Gawler is The Golden Fleece Hotel.
Retrieving results for Index 241: Mareeba.
Closest hotel in Mareeba is Mareeba Motor Inn.
Retrieving results for Index 270: Port Augusta.
Closest hotel in Port Augusta is The Hotel Flinders.
Retrieving results for Index 390: Dhidhdhoo.
Closest hotel in Dhidhdhoo is DhidhdhooINN, BeachView.
Retrieving results for Index 430: Kuching.
Closest hotel in Kuching is The Waterfront Hotel.
Retrie

In [91]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
20,Alyangula,AU,-13.8483,136.4192,Groote Eylandt Lodge
39,Kavaratti,IN,10.5669,72.6420,Bismillah Hotel And Lodging
73,Acapulco de Juárez,MX,16.8634,-99.8901,Hotel Krystal Beach Acapulco
79,Porto-Novo,BJ,6.4965,2.6036,Hotel Les Oliviers
107,Ambon City,ID,-3.6954,128.1814,Swiss-Belhotel Ambon
231,Gawler,AU,-34.6000,138.7333,The Golden Fleece Hotel
241,Mareeba,AU,-17.0000,145.4333,Mareeba Motor Inn
270,Port Augusta,AU,-32.5000,137.7667,The Hotel Flinders
390,Dhidhdhoo,MV,6.8833,73.1000,"DhidhdhooINN, BeachView"
430,Kuching,MY,1.5500,110.3333,The Waterfront Hotel


In [95]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row)
              for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]



In [105]:
# add marker layer on to heat map
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)
plt.savefig('images\hotel_markers.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 640x480 with 0 Axes>